# Step by step processing data
1. Xoá các hàng có cột Bathroom có giá trị >10
2. Xoá các giá trị NaN ở cột Price
3. Điền các giá trị thiếu ở cột Carpet Area, Current Floor
4. Thay đổi giá trị of của cột Total Floor bằng giá trị Median của cột đó

# Import libraries

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [80]:
# đọc dữ liệu
df = pd.read_csv("/Users/nguyengiabao/Documents/house_price/data/Pre_data.csv")
df

/var/folders/kn/x5nhf1_j5257w0rs737fn3bh0000gn/T/ipykernel_52461/1749871991.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/nguyengiabao/Documents/house_price/data/Pre_data.csv")


,Carpet Area,Unit,Current Floor,Total Floor,Bathroom,Price
0,500.0,sqft,10.0,11,1,6000.0
1,473.0,sqft,3.0,22,2,13799.0
2,779.0,sqft,10.0,29,2,17500.0
3,530.0,sqft,1.0,3,1,NaN
4,635.0,sqft,20.0,42,2,18824.0
...,...,...,...,...,...,...
187526,NaN,NaN,2.0,4,3,3225.0
187527,NaN,NaN,4.0,6,3,3274.0
187528,1250,sqft,1.0,3,3,4343.0
187529,NaN,NaN,2.0,2,2,4231.0


In [81]:
# kiểm tra cột và xoá khoảng trắng
df.columns = df.columns.str.strip()  # Xoá khoảng trắng đầu/cuối tên cột


In [82]:
# Xoá các hàng có giá trị NaN ở cột Price
df = df.dropna(subset=["Price"])

# Chuyển cột Bathroom sang dạng số ( vì Bathroom đang ở dạng chuỗi ">10")
df["Bathroom"] = pd.to_numeric(df["Bathroom"], errors="coerce")

# Xóa các hàng có Bathroom > 10
df = df[df["Bathroom"] <= 10]

print(df)


       Carpet Area   Unit  Current Floor Total Floor  Bathroom    Price
0            500.0   sqft           10.0          11       1.0   6000.0
1            473.0   sqft            3.0          22       2.0  13799.0
2            779.0   sqft           10.0          29       2.0  17500.0
4            635.0   sqft           20.0          42       2.0  18824.0
5              NaN    NaN            2.0           7       1.0   6618.0
...            ...    ...            ...         ...       ...      ...
187526         NaN    NaN            2.0           4       3.0   3225.0
187527         NaN    NaN            4.0           6       3.0   3274.0
187528        1250   sqft            1.0           3       3.0   4343.0
187529         NaN    NaN            2.0           2       2.0   4231.0
187530         NaN    NaN            5.0          13       4.0   6162.0

[169084 rows x 6 columns]


/var/folders/kn/x5nhf1_j5257w0rs737fn3bh0000gn/T/ipykernel_52461/1193872131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Bathroom"] = pd.to_numeric(df["Bathroom"], errors="coerce")


In [83]:
# Chuyển cột " Carpet Area" sang dạng số
df["Carpet Area"] = pd.to_numeric(df["Carpet Area"], errors="coerce")
print(df["Carpet Area"].dtype)

# Đơn vị của cột "Carpet Area"
print(df["Unit"].unique())

float64
[' sqft' nan ' sqm' ' sqyrd' ' acre' ' bigha' ' marla' ' kanal']


In [84]:
# Đổi đơn vị tính Carpet Area về cùng 1 đơn vị
# Tại sao lại xuất hiện sqft hay sqm đến 2 lần là vì do khoảng trắng, tương tự với giá trị khác
conversion_factors = {
    "sqft": 1, " sqft": 1,  
    "sqm": 10.764, " sqm": 10.764,  
    "acre": 43560, " acre": 43560,  
    "sqyrd": 9, " sqyrd": 9,  
    "marla": 272.25, " marla": 272.25,  
    "kanal": 5445, " kanal": 5445,  
    "bigha": 17427, " bigha": 17427
}

# Loại bỏ khoảng trắng thừa
df["Unit"] = df["Unit"].str.strip()
# Chuyển đổi diện tích khi tính toán
df["Carpet Area"] = df["Carpet Area"] * df["Unit"].map(conversion_factors)

print(df["Carpet Area"])


0          500.0
1          473.0
2          779.0
4          635.0
5            NaN
           ...  
187526       NaN
187527       NaN
187528    1250.0
187529       NaN
187530       NaN
Name: Carpet Area, Length: 169084, dtype: float64


In [85]:
# Xoá cột Unit 
df.drop(columns=["Unit"], inplace=True)
print(df)

        Carpet Area  Current Floor Total Floor  Bathroom    Price
0             500.0           10.0          11       1.0   6000.0
1             473.0            3.0          22       2.0  13799.0
2             779.0           10.0          29       2.0  17500.0
4             635.0           20.0          42       2.0  18824.0
5               NaN            2.0           7       1.0   6618.0
...             ...            ...         ...       ...      ...
187526          NaN            2.0           4       3.0   3225.0
187527          NaN            4.0           6       3.0   3274.0
187528       1250.0            1.0           3       3.0   4343.0
187529          NaN            2.0           2       2.0   4231.0
187530          NaN            5.0          13       4.0   6162.0

[169084 rows x 5 columns]


In [86]:
# điền giá trị trung vị với Carpet Area
imputer = SimpleImputer(strategy="median")
imputer.fit(df[["Carpet Area"]])
df[["ImputedCarpet_Area"]] = imputer.transform(df[["Carpet Area"]])
print(df["ImputedCarpet_Area"])

0          500.0
1          473.0
2          779.0
4          635.0
5         1100.0
           ...  
187526    1100.0
187527    1100.0
187528    1250.0
187529    1100.0
187530    1100.0
Name: ImputedCarpet_Area, Length: 169084, dtype: float64


In [87]:
# điền gía trị trung vị với Current Floor
imputer = SimpleImputer(strategy="median")
imputer.fit(df[["Current Floor"]])
df[["ImputedCurrent_Floor"]] = imputer.transform(df[["Current Floor"]])
print(df["ImputedCurrent_Floor"])


0         10.0
1          3.0
2         10.0
4         20.0
5          2.0
          ... 
187526     2.0
187527     4.0
187528     1.0
187529     2.0
187530     5.0
Name: ImputedCurrent_Floor, Length: 169084, dtype: float64


In [88]:
# Chuyển đổi of thành giá trị trung bình của cột của nó
df["Total Floor"] = df["Total Floor"].replace("of", np.nan)

# Chuyển đổi sang dạng số
df["Total Floor"] = pd.to_numeric(df["Total Floor"], errors="coerce")

# Điền NaN bằng phần nguyên của giá trị trung bình
df["Total Floor"].fillna(int(df["Total Floor"].mean()), inplace=True)

print(df["Total Floor"])

0         11.0
1         22.0
2         29.0
4         42.0
5          7.0
          ... 
187526     4.0
187527     6.0
187528     3.0
187529     2.0
187530    13.0
Name: Total Floor, Length: 169084, dtype: float64


/var/folders/kn/x5nhf1_j5257w0rs737fn3bh0000gn/T/ipykernel_52461/690359594.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Total Floor"].fillna(int(df["Total Floor"].mean()), inplace=True)


In [ ]:
# kiểm tra Data Frame 
df.dropna(columns=["Carpet Area", "Current Floor"], inplace=True)
print(df)

        Total Floor  Bathroom    Price  ImputedCarpet_Area  \
0              11.0       1.0   6000.0               500.0   
1              22.0       2.0  13799.0               473.0   
2              29.0       2.0  17500.0               779.0   
4              42.0       2.0  18824.0               635.0   
5               7.0       1.0   6618.0              1100.0   
...             ...       ...      ...                 ...   
187526          4.0       3.0   3225.0              1100.0   
187527          6.0       3.0   3274.0              1100.0   
187528          3.0       3.0   4343.0              1250.0   
187529          2.0       2.0   4231.0              1100.0   
187530         13.0       4.0   6162.0              1100.0   

        ImputedCurrent_Floor  
0                       10.0  
1                        3.0  
2                       10.0  
4                       20.0  
5                        2.0  
...                      ...  
187526                   2.0  
187527 

In [96]:
df.describe()

,Total Floor,Bathroom,Price,ImputedCarpet_Area,ImputedCurrent_Floor
count,169084.000000,169084.000000,1.690840e+05,1.690840e+05,169084.000000
mean,8.904953,2.445122,7.592008e+03,1.751470e+03,4.504584
std,7.489324,0.843642,2.730193e+04,1.728946e+05,4.654983
min,1.000000,1.000000,0.000000e+00,1.000000e+00,-2.000000
25%,4.000000,2.000000,4.296000e+03,1.000000e+03,2.000000
50%,6.000000,2.000000,6.039000e+03,1.100000e+03,3.000000
75%,11.000000,3.000000,9.524000e+03,1.200000e+03,6.000000
max,200.000000,10.000000,6.700000e+06,6.534000e+07,200.000000


In [97]:
# Lưu dữ liệu đã xử lý vào file CSV mới
df.to_csv("/Users/nguyengiabao/Documents/house_price/data/Pretraining_data.csv", index=False)
